In [29]:
import json
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

In [36]:
!curl localhost:9200

{
  "name" : "es01",
  "cluster_name" : "es-docker-cluster",
  "cluster_uuid" : "esmw9d6xTkay_4OD0qAXAA",
  "version" : {
    "number" : "7.14.0",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "dd5a0a2acaa2045ff9624f3729fc8a6f40835aa1",
    "build_date" : "2021-07-29T20:49:32.864135063Z",
    "build_snapshot" : false,
    "lucene_version" : "8.9.0",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [30]:
with open("../data.json") as f:
    documents = json.load(f)
    
print(len(documents))
print(documents[0])

401
{'section_title': 'Summary', 'text': 'Pandemic,Summary\nA pandemic (from Greek πᾶν, pan, "all" and δῆμος, demos, "people") is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people. A widespread endemic disease with a stable number of infected people is not a pandemic. Widespread endemic diseases with a stable number of infected people such as recurrences of seasonal influenza are generally excluded as they occur simultaneously in large regions of the globe rather than being spread worldwide.\nThroughout human history, there have been a number of pandemics of diseases such as smallpox and tuberculosis. The most fatal pandemic in recorded history was the Black Death (also known as The Plague), which killed an estimated 75–200 million people in the 14th century. The term was not used yet but was for later pandemics including the 1918 influenza pandemic (Spanish flu). Current p

In [31]:
es = Elasticsearch()

In [32]:
def gendata(documents):
    for document in documents:
        yield {
            '_op_type': 'create',
            "_index": "cdc",
            "_source" : document
        }
bulk(es, gendata(documents))

(401, [])

In [33]:
# es.indices.get_alias("*")
# es.cat.count('cdc', params={"format": "json"})

# check if an index exists and returns number of documents in the index
def check_existed(index):
    indices = es.indices.get_alias("*")
    if index in indices:
        result = es.cat.count(index, params={"format": "json"})
        return (True, result[0]['count'])
    else:
        return (False, 0)

In [38]:
existed, doc_count = check_existed('cdc')
print("Index 'cdc' existed? ", existed)
print("Number of Documented =", doc_count)

Index 'cdc' existed?  True
Number of Documented = 401


/Users/jiunjiunma/anaconda3/envs/py38/lib/python3.8/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: this request accesses system indices: [.kibana_7.14.0_001, .kibana_task_manager_7.14.0_001, .tasks, .apm-agent-configuration, .apm-custom-link], but in a future major version, direct access to system indices will be prevented by default
  warnings.warn(message, category=ElasticsearchWarning)


In [39]:
!docker-compose down

Stopping es03  ... 
Stopping es01  ... 
Stopping kib01 ... 
Stopping es02  ... 
ping kib01 ... doneRemoving es03  ... 
Removing es01  ... 
Removing kib01 ... 
Removing es02  ... 
Removing network search-tool-3_elastic
